In [289]:
import pandas as pd


In [499]:
assays_long = pd.read_csv('assays_long.csv')

# Proportion stacked bar plot:

In [500]:
###Count up conditions using groupby:
numActive = assays_long.groupby(['instance_id', 'pref_name']).apply( 
    lambda grouped_df: (grouped_df['outcome'].str.contains('Active').any())).values.sum()
numInactive = assays_long.groupby(['instance_id', 'pref_name']).apply( 
    lambda grouped_df: (grouped_df['outcome'].str.contains('Inactive').any())).values.sum()

numBoth = assays_long.groupby(['instance_id', 'pref_name']).apply( 
    lambda grouped_df: ((grouped_df['outcome'].str.contains('Inactive').any()) & 
                        (grouped_df['outcome'].str.contains('Active').any()))).values.sum()

numNone = assays_long.groupby(['instance_id', 'pref_name']).apply( 
    lambda grouped_df: (~(grouped_df['outcome'].str.contains('Inactive').any()) & 
                        ~(grouped_df['outcome'].str.contains('Active').any()))).values.sum()

In [501]:
###Or count up conditions the boring way:
y=0
n=0
b=0
neither=0
instance_ids = assays_long.drop_duplicates(['instance_id', 'pref_name'])['instance_id']
pref_names = assays_long.drop_duplicates(['instance_id', 'pref_name'])['pref_name']
for i, name in zip(instance_ids, pref_names):
    mask = (assays_long['pref_name']==name) & (assays_long['instance_id']==i)
    group = assays_long[mask]
    outcomes = list(group['outcome'])
    if ('Active' in outcomes) and ('Inactive' in outcomes):
        b+=1
    elif ('Active' in outcomes):
        y+=1
    elif ('Inactive' in outcomes):
        n+=1
    else:
        neither+=1
        

In [502]:
assays_long.groupby(['instance_id', 'pref_name']).apply( 
    lambda grouped_df: (~(grouped_df['outcome'].str.contains('Inactive').any()) & 
                        ~(grouped_df['outcome'].str.contains('Active').any()))).values.sum()

667

In [503]:
#Verify both ways are the same:
print(f'Active: {y}')
print(f'Inactive: {n}')
print(f'Both: {b}')
print(f'None: {neither}')

Active: 288
Inactive: 45
Both: 12
None: 667


In [504]:
#Both ways are the same:
print(f'Active: {numActive-numBoth}')
print(f'Inactive: {numInactive-numBoth}')
print(f'Both: {numBoth}')
print(f'None: {numNone}')

Active: 288
Inactive: 45
Both: 12
None: 667


In [505]:
df = pd.DataFrame(columns=['results','activity', 'number'])
df.loc[0]=['Results', 'Active', numActive]
df.loc[1]=['Results', 'Inactive', numInactive]
df.loc[2]=['Results', 'Both', numBoth]
df.loc[3]=['Results', 'Unknown', numNone ]
df['proportion'] = df['number']/df['number'].sum()
df

,results,activity,number,proportion
0,Results,Active,300,0.289575
1,Results,Inactive,57,0.0550193
2,Results,Both,12,0.011583
3,Results,Unknown,667,0.643822


In [506]:
chart_one = alt.Chart(df).transform_joinaggregate(
        Total='sum(number)'
    ).transform_calculate(PercentOfTotal="datum.number / datum.Total").mark_bar().encode(
    x=alt.X('PercentOfTotal:Q'),#,sort='descending'),
    y=alt.Y('results', title=''),
    color=alt.Color('activity', title='Activity type'),
    order=alt.Order('activity', sort='ascending')
).add_selection(
    selector
)

chart_two = alt.Chart(df.iloc[:3]).transform_joinaggregate(
        Total='sum(number)'
    ).transform_calculate(PercentOfTotal="datum.number / datum.Total").mark_bar().encode(
    x=alt.X('PercentOfTotal:Q'),#,sort='descending'),
    y=alt.Y('results', title=''),
    color=alt.Color('activity', title='Activity type'),
    order=alt.Order('activity', sort='ascending')
).add_selection(
    selector
)

chart_one&chart_two


alt.VConcatChart(...)

In [507]:
####Interactive version:
# brush = alt.selection(type='interval', encodings=['x'])

# base = alt.Chart(df).transform_joinaggregate(
#         Total='sum(number)'
#     ).transform_calculate(PercentOfTotal="datum.number / datum.Total").mark_bar().encode(
#     #x='number',
#     x='PercentOfTotal:Q',
#     y='results',
#     color='activity')

# upper = base.encode(
#     alt.X('PercentOfTotal:Q', scale=alt.Scale(domain=brush)))

# lower = base.add_selection(brush)

# lower&upper 


# Target type barplot:


In [508]:
out = assays_long.groupby(['instance_id', 'pref_name']).apply( 
    lambda grouped_df: (~(grouped_df['outcome'].str.contains('Inactive').any()) & 
                        (grouped_df['outcome'].str.contains('Active').any())))

positive_targets=[i[1] for i, v in zip(out.index, out.values) if v==True]
all_pos_targets = pd.DataFrame(out[out==True]).reset_index().drop([0],axis=1)
pos_targets_gb= all_pos_targets.groupby(['pref_name']).count().reset_index().sort_values(by='instance_id', ascending=False)
pos_targets_gb.columns=['pref_name', 'count']

In [509]:
##Do the count aggregation within altair. 
##Can't figure out how to get it ordered by 'count' for some reason.
chart = alt.Chart(all_pos_targets).mark_bar()

chart = chart.encode(
    x = alt.X('pref_name', sort='-y'),
    y = alt.Y('count()'), 
)#.configure_axisBottom(labelAngle=-45)

In [510]:
(chart_one&chart_two&chart).configure_axisBottom(labelAngle=-45)

alt.VConcatChart(...)

# All together now:

In [511]:
chart_one = alt.Chart(df).transform_joinaggregate(
        Total='sum(number)'
    ).transform_calculate(PercentOfTotal="datum.number / datum.Total").mark_bar().encode(
    x=alt.X('PercentOfTotal:Q'),#,sort='descending'),
    y=alt.Y('results', title='',axis=alt.Axis(labels=False)),
    color=alt.Color('activity', title='Activity type'),
    order=alt.Order('activity', sort='ascending')
).add_selection(
    selector
).properties(height=30, width=550)

chart_two = alt.Chart(df.iloc[:3]).transform_joinaggregate(
        Total='sum(number)'
    ).transform_calculate(PercentOfTotal="datum.number / datum.Total").mark_bar().encode(
    x=alt.X('PercentOfTotal:Q', title='Hi'),#,sort='descending'),
    y=alt.Y('results', title='',axis=alt.Axis(labels=False)),
    color=alt.Color('activity', title='Activity type'),
    order=alt.Order('activity', sort='ascending')
).add_selection(
    selector
).properties(height=30, width=550)

chart = alt.Chart(all_pos_targets).mark_bar()

chart = chart.encode(
    x = alt.X('pref_name', sort='-y', title='Target preferred name'),
    y = alt.Y('count()'), 
).properties(height=200,width=700)

(chart_one&chart_two&chart
).configure_axisBottom(labelAngle=-35).configure_header(
    titleFontSize=40, 
    labelFontSize=40
).configure_title()

alt.VConcatChart(...)

In [512]:
def parse_group(group):
    inactive = group['outcome'].str.contains('Inactive').any()
    active = group['outcome'].str.contains('Active').any()
    if inactive and active:
        return 'Both'
    elif active and ~inactive:
        return 'Active'
    elif inactive and ~active:
        return 'Inactive'
    else:
        return False
    
    
out = assays_long.groupby(['instance_id', 'pref_name']).apply(parse_group)
out_df= pd.DataFrame(out[out!=False]).reset_index().drop('instance_id',axis=1)
out_df.columns=['pref_name', 'activity']

In [556]:
chart_one = alt.Chart(df).transform_joinaggregate(
        Total='sum(number)'
    ).transform_calculate(PercentOfTotal="datum.number / datum.Total").mark_bar().encode(
    x=alt.X('PercentOfTotal:Q', title='Proportion'),#,sort='descending'),
    y=alt.Y('results', title='All records',axis=alt.Axis(labels=False)),
    color=alt.Color('activity', title='Activity type'),
    order=alt.Order('activity', sort='ascending')
).add_selection(
    selector
).properties(height=30, width=550)

chart_two = alt.Chart(df.iloc[:3]).transform_joinaggregate(
        Total='sum(number)'
    ).transform_calculate(PercentOfTotal="datum.number / datum.Total").mark_bar().encode(
    x=alt.X('PercentOfTotal:Q', title='Proportion'),#,sort='descending'),
    y=alt.Y('results', title='Records with evidence',axis=alt.Axis(labels=False)),
    color=alt.Color('activity', title='Activity type'),
    order=alt.Order('activity', sort='ascending')
).add_selection(
    selector
).properties(height=30, width=550)








ch = alt.Chart(out_df).mark_bar()
ch = ch.encode(
    x = alt.X('pref_name', sort='-y', title='Target preferred name'),
    y = alt.Y('count()'),
    color=alt.Color('activity'),
    order=alt.Order('activity', sort='ascending'),
).properties(height=200,width=550)


(chart_one&chart_two&ch
).configure_axisBottom(labelAngle=-35).configure_header(
    titleFontSize=40, 
    labelFontSize=40
).configure_axisY(
    titleAngle=0,
    titleAlign="left",
    titleY=-10,
    titleX=-10,
    titleFontSize=16,
)

alt.VConcatChart(...)

In [564]:
chart_one = alt.Chart(df).transform_joinaggregate(
        Total='sum(number)'
    ).transform_calculate(PercentOfTotal="datum.number / datum.Total").mark_bar().encode(
    y=alt.Y('PercentOfTotal:Q', title='Proportion'),#,sort='descending'),
    x=alt.X('results', title='All records',axis=alt.Axis(labels=False)),
    color=alt.Color('activity', title='Activity type'),
    order=alt.Order('activity', sort='ascending')
).add_selection(
    selector
).properties(height=550, width=30)

chart_two = alt.Chart(df.iloc[:3]).transform_joinaggregate(
        Total='sum(number)'
    ).transform_calculate(PercentOfTotal="datum.number / datum.Total").mark_bar().encode(
    y=alt.Y('PercentOfTotal:Q', title='Proportion'),#,sort='descending'),
    x=alt.X('results', title='Records with evidence',axis=alt.Axis(labels=False)),
    color=alt.Color('activity', title='Activity type'),
    order=alt.Order('activity', sort='ascending')
).add_selection(
    selector
).properties(height=550, width=30)








ch = alt.Chart(out_df).mark_bar()
ch = ch.encode(
    y = alt.Y('pref_name', sort='-x', title='Target preferred name'),
    x = alt.X('count()'),
    color=alt.Color('activity'),
    order=alt.Order('activity', sort='ascending'),
).properties(height=550,width=200)


(chart_one|chart_two|ch
).configure_axisBottom(labelAngle=-35).configure_header(
    titleFontSize=40, 
    labelFontSize=40
).configure_axisY(
    titleAngle=0,
    titleAlign="left",
    titleY=-10,
    titleX=-10,
    titleFontSize=16,
)

alt.HConcatChart(...)